# Camera run test
Open camera, s to save, q to quit

In [ ]:
import pyrealsense2 as rs
import numpy as np
import cv2
import os

# Configure RealSense pipeline
pipeline = rs.pipeline()
config = rs.config()
colorizer = rs.colorizer() # Create colorizer for depth map

# Create folder to save images
SAVE_DIR = "images/test"
os.makedirs(SAVE_DIR, exist_ok=True)

# Enable color and depth streams
config.enable_stream(rs.stream.color, 640, 480, rs.format.bgr8, 30)
config.enable_stream(rs.stream.depth, 640, 480, rs.format.z16, 30) # depth in 16bit format

# Start streaming
pipeline.start(config)
frame_count = 0

try:
    while True:
        # wait for a pair of depth and color
        frames = pipeline.wait_for_frames()
        color_frame = frames.get_color_frame()
        depth_frame = frames.get_depth_frame()
        if not color_frame or not depth_frame:
            continue

        # Convert images to numpy arrays
        color_image = np.asanyarray(color_frame.get_data())
        depth_image = np.asanyarray(colorizer.colorize(depth_frame).get_data())
        depth_map = np.asanyarray(depth_frame.get_data())
        # print(depth_map.dtype)
        # print(depth_map.min(), depth_map.max())
        # print(f"Depth map values: {depth_map}")
        # print(f"Depth image values: {depth_image}")
        
        
        # Stack images side by side
        images = np.hstack((color_image, depth_image))
        # Show images
        cv2.imshow('RealSense RGB + Depth', images)

        key = cv2.waitKey(1) & 0xFF # Get key every 1ms
        # Press 'q' to exit
        if key == ord('q'):
            break
        # Press 's' to save images
        elif key == ord('s'):
            color_path = os.path.join(SAVE_DIR, f"color_{frame_count:04d}.png")
            depth_path = os.path.join(SAVE_DIR, f"depth_{frame_count:04d}.png")
            depth_map_path = os.path.join(SAVE_DIR, f"depth_{frame_count:04d}.npy")
            cv2.imwrite(color_path, color_image)
            cv2.imwrite(depth_path, depth_image)
            np.save(depth_map_path, depth_map)
            print(f"[INFO] Saved color to {color_path} depth to {depth_path} and depth map to {depth_map_path}")
            frame_count += 1
finally:
    # Stop streaming
    pipeline.stop()
    cv2.destroyAllWindows()


[INFO] Saved color to images/test/color_0000.png depth to images/test/depth_0000.png and depth map to images/test/depth_0000.npy


# Print out camera intrinsics

In [2]:
import pyrealsense2 as rs

# Create pipeline
pipeline = rs.pipeline()
config = rs.config()
config.enable_stream(rs.stream.depth, 640, 480, rs.format.z16, 30)
config.enable_stream(rs.stream.color, 640, 480, rs.format.bgr8, 30)

# Start streaming
profile = pipeline.start(config)

# Get stream profile and camera intrinsics
depth_profile = profile.get_stream(rs.stream.depth)
intrinsics = depth_profile.as_video_stream_profile().get_intrinsics()

# Get depth scale
profile = pipeline.get_active_profile()
depth_sensor = profile.get_device().first_depth_sensor()
depth_scale = depth_sensor.get_depth_scale()

print(f"Depth Scale: {depth_scale}")
print("Width:", intrinsics.width)
print("Height:", intrinsics.height)
print("fx:", intrinsics.fx)
print("fy:", intrinsics.fy)
print("ppx (cx):", intrinsics.ppx)
print("ppy (cy):", intrinsics.ppy)
print("Distortion Model:", intrinsics.model)
print("Distortion Coefficients:", intrinsics.coeffs)

# Stop streaming
pipeline.stop()


Depth Scale: 0.0010000000474974513
Width: 640
Height: 480
fx: 386.8893737792969
fy: 386.8893737792969
ppx (cx): 321.6220703125
ppy (cy): 240.8651123046875
Distortion Model: distortion.brown_conrady
Distortion Coefficients: [0.0, 0.0, 0.0, 0.0, 0.0]
